In [2]:
import pandas as pd 
import json
import os
import numpy as np
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import punkt
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix
import string
from nltk.probability import FreqDist
import seaborn as sns
pd.options.display.max_rows = 999
pd.options.display.max_columns = 30
import lexnlp as lnlp
import src
from src import *
import importlib
import unidecode as unidecode
importlib.reload(src)
%matplotlib inline

# NLP Feature Engineering 

In [3]:
final_df = pd.read_csv("../data/Final_Merge.csv")#importing the text data I cleaned
final_df.head()#checking it out

,case,text,target,lib_or_con,majVotes
0,352us282,may it please the court this case be here on a...,1,2.0,6
1,353us586,mr chief justice if the court please when the ...,1,2.0,4
2,352us249,if the court please you might wait just a mome...,0,2.0,5
3,354us147,mr chief justice if the court please this be a...,0,2.0,5
4,352us407,mr chief justice may it please the court this ...,1,1.0,6


## Tokenizing all that sweet sweet text


In [4]:
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9!]+')#instaniating a reg ex tokenizer

final_df.text = final_df.text.apply(lambda x: tokenizer.tokenize(x)) #applying it to the text column

## Lemmatizing

In [5]:
#using a custom function to lemmatize my text data
final_df.text = final_df.text.apply(lambda x: src.lemm_text(x))

In [6]:
#checking my work
final_df.head()

,case,text,target,lib_or_con,majVotes
0,352us282,"[may, it, please, the, court, this, case, be, ...",1,2.0,6
1,353us586,"[mr, chief, justice, if, the, court, please, w...",1,2.0,4
2,352us249,"[if, the, court, please, you, might, wait, jus...",0,2.0,5
3,354us147,"[mr, chief, justice, if, the, court, please, t...",0,2.0,5
4,352us407,"[mr, chief, justice, may, it, please, the, cou...",1,1.0,6


# Modeling 


I will be using accuracy as my metric as this model will have little actionable impact, its not important to maximise other metrics. The more accurate the model the better a data point it will be for legal prognosticators.

## Train Test Split and vectorizing with TFIDF scores

I used sklearn to get the tfidf scores for my data. I tried a couple of different ngram ranges and 1-3 seemed to be the best.

In [8]:
#to use the tfidf model as I know it, I rejoined the lemmatized words into one string
final_df.text = final_df.text.apply(lambda x: ','.join(x)) 
final_df.text =final_df.text.apply(lambda x: x.replace(',',' '))

In [9]:
X = final_df.text#assigning my features
Y = final_df.target#and my targets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [30]:
#creating stopwords, I included some legal ones, I could have done this in my src file 
#and imported them, but it was easier to expirement with different options by generating
#them within the notebook
sw_list = stopwords.words('english')
sw_list += list(string.punctuation)
sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘', '©',
        'said', 'one', 'com','-', '–', '—', 'co', 'wa', 'ha', '1', 'amp',
        'court', 'would', 'case', 'say', 'think''may', 'it', 'please', 'the', 'court', 'justice', 'thank', 'you', 'mrs', 'chief', 'honor'] 
sw_set = set(sw_list)

In [31]:
#train test split
X_train_lem, X_test_lem, y_train_lem, y_test_lem = train_test_split(X, Y, test_size=0.6, random_state=34)
#vectorizing with TFIDF scores
tfidf = TfidfVectorizer(ngram_range= (1,3), stop_words= sw_set)#I tried different ranges of ngrams and 1-3 was the best
#transforming the data 
tfidf_data_train_lem = tfidf.fit_transform(X_train_lem)
tfidf_data_test_lem = tfidf.transform(X_test_lem)

### top ten TFIDF scores

In [32]:
indices = np.argsort(tfidf.idf_)[::-1]#grabbing the top ten tfidf score indices
features = tfidf.get_feature_names() #getting the feature names
top_n = 10
top_features = [features[i] for i in indices[:top_n]] # matching them up 
print (top_features)

['zwiener question obscenity', 'ford versus wainwright', 'ford summary underlying', 'ford summary reason', 'ford summary', 'ford summarize overall', 'ford summarize', 'ford suggest whether', 'ford suggest', 'ford submit rule']


The main interesting word here is forbid, it's featured in nine of the top ten tri grams, bigrams or words. Furthermore, almost every high tfidf word is a trigram with only three bigrams in the top ten. Obscenity and free speech show up which are interrelated issues and were also more likely to be winning issues for the petitioners. 

## Untuned RFC (baseline)

I will use an RFC as my baseline model. It's generally good for NLP classification tasks and not overly difficult to implement untuned.

In [33]:
#instantiating the model
rf_classifier_lem = RandomForestClassifier(n_estimators=10, random_state=0, class_weight= 'balanced')


In [34]:
#fitting
rf_classifier_lem.fit(tfidf_data_train_lem, y_train_lem)
#generating test predictions
rf_test_preds_lem = rf_classifier_lem.predict(tfidf_data_test_lem)
#generating train predictions
rf_train_preds_lem = rf_classifier_lem.predict(tfidf_data_train_lem)

### Evaulating test predictions

In [16]:
from sklearn.metrics import f1_score
rf_acc_score_lem = accuracy_score(y_test_lem, rf_test_preds_lem)#accuracy score
rf_f1_score_lem = f1_score(y_test_lem, rf_test_preds_lem)#F1 score
print('Random Forest with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(rf_acc_score_lem))
print()
print("F1 Score: {:.4}".format(rf_f1_score_lem))

Random Forest with Lemmatization Features
Testing Accuracy: 0.5136

F1 Score: 0.5625


In [17]:
confusion = confusion_matrix(y_test_lem, rf_test_preds_lem)#generating a confusion matrix
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
#printing it in the format which I find most readable, rather than the graphic version
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 461
True Negatives: 296
 False Positives: 306
 False Negatives: 411



### Evaulating training predictions

In [20]:
rf_acc_score_lem_t = accuracy_score(y_train_lem, rf_train_preds_lem)#accuracy score
rf_f1_score_lem_t = f1_score(y_train_lem, rf_train_preds_lem)# F1 score
print('Random Forest with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(rf_acc_score_lem_t))
print()
print("F1 Score: {:.4}".format(rf_f1_score_lem_t))

Random Forest with Lemmatization Features
Testing Accuracy: 0.9878

F1 Score: 0.9894


In [21]:
confusion = confusion_matrix(y_train_lem, rf_train_preds_lem) #generating a confusion matrix
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
#printing it in the format which I find most readable, rather than the graphic version
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 558
True Negatives: 412
 False Positives: 7
 False Negatives: 5



### Analysis

Seems like the baseline model is way way way overfit, I will try some different models and see if they have similar problems, then try a gridsearch cv to try to adress this problem.

## Multinomial Naive Bayes Classifier

In [22]:
from sklearn.naive_bayes import MultinomialNB 

In [23]:
nb_classifier = MultinomialNB(alpha = .1)#instantiating a multinomial naive bayes model 

In [24]:
nb_classifier.fit(tfidf_data_train_lem, y_train_lem) #fitting


MultinomialNB(alpha=0.1)

In [25]:
nb_test_preds_lem = nb_classifier.predict(tfidf_data_test_lem)
nb_train_preds_lem = nb_classifier.predict(tfidf_data_train_lem)


In [26]:
nb_acc_score_lem = accuracy_score(y_test_lem, nb_test_preds_lem)
nb_f1_score_lem = f1_score(y_test_lem, nb_test_preds_lem)

In [27]:
print('Naive Bayes with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(nb_acc_score_lem))
print()
print("F1 Score: {:.4}".format(nb_f1_score_lem))

Naive Bayes with Lemmatization Features
Testing Accuracy: 0.5909

F1 Score: 0.7429


The Naive Bayes model did a little better on the testing data but not a by a huge margin. It is about the same as predicting the dominant class wil always win (petitioner. 

In [123]:
confusion = confusion_matrix(y_test_lem, rf_test_preds_lem)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 519
True Negatives: 284
 False Positives: 318
 False Negatives: 353



In [28]:
nb_acc_score_lem_t = accuracy_score(y_train_lem, nb_train_preds_lem)
nb_f1_score_lem_t = f1_score(y_train_lem, nb_train_preds_lem)

In [29]:
print('Naive Bayes with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(nb_acc_score_lem_t))
print()
print("F1 Score: {:.4}".format(nb_f1_score_lem))

Naive Bayes with Lemmatization Features
Testing Accuracy: 1.0

F1 Score: 0.7429


In [126]:
confusion = confusion_matrix(y_train_lem, nb_train_preds_lem)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 563
True Negatives: 419
 False Positives: 0
 False Negatives: 0



Still having the same drastic overfitting issues with this model 

### Top Ten Most Imporant Features

In [25]:
feature_names = tfidf.get_feature_names()
top10 = np.argsort(nb_classifier.coef_[0])[-10:]

print(" ".join(feature_names[j] for j in top10))

honor make right well state think say would case court


These are largely the same as the most common words I found in my EDA. I thought that with TFIDF scores and modeling there would be more differences.

## Tuned RFC

I dont have high expectations but lets see if a grid search RCF can do any better before we fully decide on using the NB classifier. 

In [75]:
from sklearn.model_selection import GridSearchCV

In [76]:
param_grid = { 
    'n_estimators': [10, 20, 50, 100],
    'criterion': ['gini'],
    'max_depth': range(2,10),
    'max_features': ['auto']
}

In [77]:
grid_tree =GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

In [78]:
grid_tree.fit(tfidf_data_train_lem, y_train_lem)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  8.8min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': range(2, 10),
                         'max_features': ['auto'],
                         'n_estimators': [10, 20, 50, 100]},
             scoring='accuracy', verbose=1)

In [85]:
grfc_preds_test =grid_tree.predict(tfidf_data_test_lem)
grfc_preds_train = grid_tree.predict(tfidf_data_train_lem)


In [81]:
accuracy_score_test = accuracy_score(grfc_preds, y_test_lem)
f1_score_test = f1_score(grfc_preds, y_test_lem)

In [83]:
print('Grid searched Random Forest Classifier with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(accuracy_score_test))
print()
print("F1 Score: {:.4}".format(f1_score_test))

Grid searched Random Forest Classifier with Lemmatization Features
Testing Accuracy: 0.5813

F1 Score: 0.7352


In [88]:
confusion = confusion_matrix(y_test_lem, grfc_preds_test)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 286
True Negatives: 0
 False Positives: 206
 False Negatives: 0



Unfortunately this model is only predicting the dominant class 

In [86]:
accuracy_score_train = accuracy_score(grfc_preds_train, y_train_lem)
f1_score_train = f1_score(grfc_preds_train, y_train_lem)

In [87]:
print('Grid searched Random Forest Classifier with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(accuracy_score_train))
print()
print("F1 Score: {:.4}".format(f1_score_train))

Grid searched Random Forest Classifier with Lemmatization Features
Testing Accuracy: 0.585

F1 Score: 0.7382


In [89]:
confusion = confusion_matrix(y_train_lem, grfc_preds_train)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 1149
True Negatives: 0
 False Positives: 815
 False Negatives: 0



It does the same on the training data.

In [80]:
grid_tree.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'max_features': 'auto',
 'n_estimators': 20}

It looks like the grid search proccess adressed the overfitting by increasing the bias by lowering the max depth and estimators. This eventually must have led to just predicting that the petitioner would win, This indicates to me that modeling based on text data may be pretty much impossible without a sequential nueral net. I will try a few more models with the straight up TFIDF score but I do not think they will be any better. 

## Support Vector Classifier

In [41]:
from sklearn.svm import SVC

In [42]:
svc = SVC(class_weight= 'balanced')

In [43]:
svc.fit(X= tfidf_data_train_lem, y= y_train_lem)

SVC(class_weight='balanced')

In [44]:
SVC_preds_test = svc.predict(tfidf_data_test_lem)
SVC_preds_train = svc.predict(tfidf_data_train_lem)

In [49]:
svc_acc_test = accuracy_score(SVC_preds_test, y_test_lem)
svc_f1_test = f1_score(SVC_preds_test, y_test_lem)

In [50]:
print('Grid searched Random Forest Classifier with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(svc_acc_test))
print()
print("F1 Score: {:.4}".format(svc_f1_test))

Grid searched Random Forest Classifier with Lemmatization Features
Testing Accuracy: 0.5672

F1 Score: 0.6577


In [52]:
confusion = confusion_matrix(y_test_lem, SVC_preds_test)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print( f'True Positives: {TP}\n'
       f'True Negatives: {TN}\n', 
      f'False Positives: {FP}\n',
      f'False Negatives: {FN}\n')

True Positives: 613
True Negatives: 223
 False Positives: 379
 False Negatives: 259



In [ ]:
print('Grid searched Random Forest Classifier with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(accuracy_score_test))
print()
print("F1 Score: {:.4}".format(f1_score_test))

Many roads lead to the same place. Every model is fairly overfit and the optimized random forest classifier which I gridsearched finds it best to just guess that the petitioner will win 100% of the cases. I'm unsure exactly why the overfitting is happenening, it might be due to the curse of dimensionality and the multitude of words within the dataset. XG boost models are supposed to be good at adressing overfitting so I will give that a shot then give up on these more straight forward TFIDF models.

# W2V

Lets see if using W2V and getting the mean vector for each argument leads to a better model.

In [486]:
vocab = df.text.map(word_tokenize) # generating all my tokens in a format good for Word2Vec

In [487]:
from gensim.models import Word2Vec

model = Word2Vec(vocab, size=100, window=5, min_count=1, workers=4) #generating the model 

model.train(vocab, total_examples=model.corpus_count, epochs= 10) # training

(237773039, 795415180)

In [490]:
wtv = dict(zip(model.wv.index2word, model.wv.syn0)) #making a dictionary for each words vector

/Users/aidancoco/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
# I will use a use a custom word2vecVectorizer class than can get the mean vector for each case then implement that in a pipeline 
#three different models, an RFC, a SVM and a simple Logistic Regression, then display the mean cross val score for
#each model over 5 folds

from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Word2Vec Vectorizer', src.W2vVectorizer(wtv)),
              ('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Word2Vec Vectorizer', src.W2vVectorizer(wtv)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([('Word2Vec Vectorizer', srr.W2vVectorizer(wtv)),
              ('Logistic Regression', LogisticRegression(max_iter= 1000))])


In [494]:
models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]

In [500]:
scores = [(name, cross_val_score(model, final_df.text, final_df['target'], cv=2).mean()) for name, model, in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [501]:
scores

[('Random Forest', 0.5565960912052117),
 ('Support Vector Machine', 0.5842833876221498),
 ('Logistic Regression', 0.5842833876221498)]

I would do more to cross validate these models, but I am starting to get deja vu from these scores, they all hit a hard limit right around where always guessing the dominant class would get you in terms of accuracy. The SVM with the W2V mean vectors for each case does the "best" but is still not good or worth pursuing from an analysis perspective.

# Conclusions

All these models trained on my language data worked rather poorly. While this is a difficult thing to predict, there was no improvement over simply guessing the dominant class everytime. I would not reccomend using NLP to predict Supreme Court Cases. Other researchers have had far more success using other factors of the case in machine learning algorithims. 